## Deliverable 2. Create a Customer Travel Destinations Map.

In [10]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [11]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yellowknife,CA,62.46,-114.35,-11.99,75,20,2.24,few clouds
1,1,Lagoa,PT,39.05,-27.98,55.90,66,51,27.51,broken clouds
2,2,Kapaa,US,22.08,-159.32,71.60,64,40,10.29,scattered clouds
3,3,Mehamn,NO,71.04,27.85,35.04,84,100,13.31,overcast clouds
4,4,Hommelvik,NO,63.41,10.79,27.00,92,0,1.12,clear sky


In [12]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 20
What is the maximum temperature you would like for your trip? 45


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Mehamn,NO,71.04,27.85,35.04,84,100,13.31,overcast clouds
4,4,Hommelvik,NO,63.41,10.79,27.00,92,0,1.12,clear sky
9,9,Ushuaia,AR,-54.80,-68.30,42.80,75,40,14.99,scattered clouds
13,13,Pulandian,CN,39.40,121.97,28.40,54,75,2.24,broken clouds
15,15,Jamestown,US,42.10,-79.24,39.00,69,90,10.29,overcast clouds
20,20,Paamiut,GL,61.99,-49.67,32.31,97,100,17.38,light snow
21,21,Punta Arenas,CL,-53.15,-70.92,44.60,65,75,13.87,broken clouds
28,28,Vezirkopru,TR,41.14,35.45,37.40,51,0,2.24,clear sky
31,31,Shelburne,CA,44.08,-80.20,41.00,83,15,14.43,few clouds
32,32,Nikolskoye,RU,59.70,30.79,33.80,80,90,11.18,overcast clouds


In [14]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             132
City                   132
Country                132
Lat                    132
Lng                    132
Max Temp               132
Humidity               132
Cloudiness             132
Wind Speed             132
Current Description    132
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Mehamn,NO,71.04,27.85,35.04,84,100,13.31,overcast clouds
4,4,Hommelvik,NO,63.41,10.79,27.00,92,0,1.12,clear sky
9,9,Ushuaia,AR,-54.80,-68.30,42.80,75,40,14.99,scattered clouds
13,13,Pulandian,CN,39.40,121.97,28.40,54,75,2.24,broken clouds
15,15,Jamestown,US,42.10,-79.24,39.00,69,90,10.29,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
661,661,Hornepayne,CA,49.21,-84.77,32.67,91,57,9.66,broken clouds
665,665,Komsomolskiy,UZ,40.43,71.72,33.80,80,11,0.54,mist
669,669,Petrozavodsk,RU,61.78,34.35,28.40,92,99,6.71,overcast clouds
674,674,Kitami,JP,43.80,143.89,33.01,74,51,8.99,broken clouds


In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Mehamn,NO,35.04,overcast clouds,71.04,27.85,
4,Hommelvik,NO,27.00,clear sky,63.41,10.79,
9,Ushuaia,AR,42.80,scattered clouds,-54.80,-68.30,
13,Pulandian,CN,28.40,broken clouds,39.40,121.97,
15,Jamestown,US,39.00,overcast clouds,42.10,-79.24,
20,Paamiut,GL,32.31,light snow,61.99,-49.67,
21,Punta Arenas,CL,44.60,broken clouds,-53.15,-70.92,
28,Vezirkopru,TR,37.40,clear sky,41.14,35.45,
31,Shelburne,CA,41.00,few clouds,44.08,-80.20,
32,Nikolskoye,RU,33.80,overcast clouds,59.70,30.79,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

    # 6c. Get latitude and longitude from DataFrame
    
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [19]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Mehamn,NO,35.04,overcast clouds,71.04,27.85,Mehamn Arctic Hotel AS
4,Hommelvik,NO,27.00,clear sky,63.41,10.79,Best Western Stav Hotel
9,Ushuaia,AR,42.80,scattered clouds,-54.80,-68.30,Albatros Hotel
13,Pulandian,CN,28.40,broken clouds,39.40,121.97,HR East Hotel
15,Jamestown,US,39.00,overcast clouds,42.10,-79.24,Hampton Inn & Suites Jamestown
...,...,...,...,...,...,...,...
661,Hornepayne,CA,32.67,broken clouds,49.21,-84.77,Jackfish River Inn
665,Komsomolskiy,UZ,33.80,mist,40.43,71.72,Ikathouse
669,Petrozavodsk,RU,28.40,overcast clouds,61.78,34.35,"Первый рыбный ресторан ""Фрегат"""
674,Kitami,JP,33.01,broken clouds,43.80,143.89,Toyoko INN Kitami Ekimae


In [23]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file)


In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))